In [ ]:
!curl https://gluon-cv.mxnet.io/_downloads/b6ade342998e03f5eaa0f129ad5eee80/mscoco.py -o "mscoco.py"


In [ ]:
!pip install gluoncv mxnet-cu100 --pre

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!pip install opencv-python

In [ ]:
!apt update  
!apt install cuda-libraries-10-0 -y

In [ ]:
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

In [23]:
!python3 mscoco.py

  0%|                                | 52209/18883654 [00:07<39:04, 8032.07KB/s]^C
Traceback (most recent call last):
  File "mscoco.py", line 53, in <module>
    download_coco(path, overwrite=args.overwrite)
  File "mscoco.py", line 38, in download_coco
    filename = download(url, path=path, overwrite=overwrite, sha1_hash=checksum)
  File "/Users/sergeyantonov/Desktop/Graduation_work/Python/lib/python3.7/site-packages/gluoncv/utils/download.py", line 79, in download
    unit='KB', unit_scale=False, dynamic_ncols=True):
  File "/Users/sergeyantonov/Desktop/Graduation_work/Python/lib/python3.7/site-packages/tqdm/std.py", line 1108, in __iter__
    for obj in iterable:
  File "/Users/sergeyantonov/Desktop/Graduation_work/Python/lib/python3.7/site-packages/requests/models.py", line 751, in generate
    for chunk in self.raw.stream(chunk_size, decode_content=True):
  File "/Users/sergeyantonov/Desktop/Graduation_work/Python/lib/python3.7/site-packages/urllib3/response.py", line 564, in st

In [2]:
from gluoncv import data, utils
from matplotlib import pyplot as plt

coco_train_dataset = data.COCODetection(root='Enter your path', splits=['instances_train2017'])
coco_val_dataset = data.COCODetection(root='Enter your path', splits=['instances_val2017'])
print('Num of training images:', len(coco_train_dataset))
print('Num of validation images:', len(coco_val_dataset))

loading annotations into memory...
Done (t=31.20s)
creating index...
index created!
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
Num of training images: 117266
Num of validation images: 4952


In [ ]:
!curl https://gluon-cv.mxnet.io/_downloads/c1a755f8157dad15da38cca607e4189e/pascal_voc.py -o "pascal_voc.py"

In [7]:
!python3 pascal_voc.py

100%|███████████████████████████████| 1952773/1952773 [12:49<00:00, 2537.58KB/s]
1386270KB [23:41, 975.50KB/s]                                                   


In [3]:
from gluoncv import data, utils
from matplotlib import pyplot as plt

voc_train_dataset = data.VOCDetection(root='Enter your path', splits=[(2007, 'trainval'), (2012, 'trainval')])
voc_val_dataset = data.VOCDetection(root='Enter your path', splits=[(2007, 'test')])
print('Num of training images:', len(voc_train_dataset))
print('Num of validation images:', len(voc_val_dataset))

Num of training images: 16551
Num of validation images: 4952


In [34]:
# fname = utils.download('https://github.com/dmlc/web-data/blob/master/gluoncv/datasets/VOCtemplate.zip?raw=true', 'VOCtemplate.zip')
# with zipfile.ZipFile(fname) as zf:
#     zf.extractall('.')
from gluoncv.data import VOCDetection
class VOCLike(VOCDetection):
    CLASSES = ['person']
    def __init__(self, root, splits, transform=None, index_map=None, preload_label=True):
        super(VOCLike, self).__init__(root, splits, transform, index_map, preload_label)

dataset = VOCLike(root='VOCdevkit', splits=((2018, 'train'),))
print('length of dataset:', len(dataset))
print('label example:')
print(dataset[0][1])

length of dataset: 2501
label example:
[]


In [4]:
import pascalVoc_converter
import os
import cv2

class PascalVOCDataset():
    def __init__(self, root_path, fix_dataset = False):
        if not os.path.exists(root_path):
            os.makedirs(root_path)
            
        self._im_path = os.path.join(root_path, "JPEGImages")
        if not os.path.exists(self._im_path):
            os.mkdir(self._im_path)
            
        self._an_path = os.path.join(root_path, "Annotations")
        if not os.path.exists(self._an_path):
            os.mkdir(self._an_path)
            
        self._im_list = os.listdir(self._im_path)
        self._im_list = list(filter(lambda x: x.endswith('.jpg'), self._im_list))
        self._an_list = os.listdir(self._an_path)
        self._an_list = list(filter(lambda x: x.endswith('.xml'), self._an_list))
        
        if fix_dataset:
            for an in self._an_list:
                if os.path.splitext(an)[0] + '.jpg' not in self._im_list:
                    os.remove(os.path.join(self._an_path, an))
            for im in self._im_list:
                if os.path.splitext(im)[0] + '.xml' not in self._an_list:
                    os.remove(os.path.join(self._im_path, im))
                    
            self._im_list = os.listdir(self._im_path)
            self._im_list = list(filter(lambda x: x.endswith('.jpg'), self._im_list))
            self._an_list = os.listdir(self._an_path)
            self._an_list = list(filter(lambda x: x.endswith('.xml'), self._an_list))
        
        self._items = [int(os.path.splitext(an)[0]) for an in self._an_list]
        
        self._last_index = 0
        if len(self._items) > 0:
            self._last_index = max(self._items)
            
        
    def addItem(self, img, anotation):
        ind = self._last_index + 1
        self._last_index = ind
        
        im_path = os.path.join(self._im_path, str(ind)+'.jpg')
        cv2.imwrite(im_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        
        writer = pascalVoc_converter.Writer(im_path, width=img.shape[1], height=img.shape[0])
 
        for bounding_box, class_id in anotation:
            x_min, y_min, x_max, y_max = bounding_box
            writer.addObject(x_min, y_min, x_max, y_max)
            
        an_path = os.path.join(self._an_path, str(ind)+'.xml')
        writer.save(an_path)
        

In [5]:
from tqdm import tqdm
def extractData(srcDataset, target_dataset, types):
    """
    srcDataset -- VisionDataset source
    target_path -- root path of new PascalVOC dataset
    types -- types for extracting
    """
    for image, label in tqdm(srcDataset):
        image = image.asnumpy()
        bounding_boxes = label[:, :4]
        class_ids = label[:, 4:5]
        target_anotation = []
        for bounding_box, class_id in zip(bounding_boxes, class_ids):
            if class_id in types:
                target_anotation.append([bounding_box, class_id])
        if len(target_anotation) > 0:
            target_dataset.addItem(image, target_anotation)

In [8]:
own_train_dataset = PascalVOCDataset('./data/ABA_Person_Dataset/train', True)
types = []
types.append(voc_train_dataset.classes.index('person'))
extractData(voc_train_dataset, own_train_dataset, types)

types = []
types.append(coco_train_dataset.classes.index('person'))
extractData(coco_train_dataset, own_train_dataset, types)

own_val_dataset = PascalVOCDataset('./data/ABA_Person_Dataset/val', True)
types = []
types.append(voc_val_dataset.classes.index('person'))
extractData(voc_val_dataset, own_val_dataset, types)

types = []
types.append(coco_val_dataset.classes.index('person'))
extractData(coco_val_dataset, own_val_dataset, types)


 17%|█▋        | 2875/16551 [00:34<02:33, 88.96it/s]


 36%|███▌      | 5999/16551 [01:09<02:25, 72.34it/s]


 56%|█████▌    | 9258/16551 [01:42<01:18, 93.21it/s]


 79%|███████▉  | 13061/16551 [02:14<00:28, 123.85it/s]


100%|██████████| 16551/16551 [02:57<00:00, 93.32it/s]

  0%|          | 13/117266 [00:00<29:59, 65.14it/s]


...

100%|██████████| 117266/117266 [36:42<00:00, 53.24it/s]

  7%|▋         | 357/4952 [00:03<00:48, 95.54it/s]


 81%|████████  | 4008/4952 [00:38<00:13, 70.59it/s]


100%|██████████| 4952/4952 [00:47<00:00, 103.87it/s]

 32%|███▏      | 1609/4952 [00:26<00:55, 60.35it/s]


 75%|███████▌  | 3721/4952 [01:03<00:20, 61.31it/s]


100%|██████████| 4952/4952 [01:27<00:00, 56.48it/s]
